<a href="https://colab.research.google.com/github/neel26desai/advanced_prompt_engineering_stategies/blob/main/Gemini_Palm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Palm2 apis are deprecated as per https://ai.google.dev/palm_docs/palm, and are replaced wit gemini api , hence using them.

In [1]:
!pip install google-ai-generativelanguage
!pip install --quiet langchain-core
!pip install --quiet langchain-google-genai
!pip install --quiet -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [2]:
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document

In [3]:

import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('gemini')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [5]:
response = llm.invoke("what is the capital of India?")

In [6]:
response.content

'The capital of India is **New Delhi**.\n'

In [7]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

## 1. Text Summarization

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant that summarizes text."),
]
long_text = '''

Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus.

'''
messages.append(HumanMessage(content=long_text))
response = llm.invoke(messages)

In [9]:
print(response.content)

Jupiter, the fifth planet from the Sun, is the Solar System's largest.  A gas giant, it's one-thousandth the mass of the Sun but outweighs all other planets combined.  Visible to the naked eye, it's been known since antiquity and is named after the Roman god.  Its brightness can even cast shadows on Earth.



## 2. Grammar Correction

In [10]:
messages = [
    SystemMessage(content="You will be provided with statements, and your task is to convert them to standard English."),
    HumanMessage(content="She no went to the market.,")
]
response = llm.invoke(messages)

In [11]:
print(response.content)

She did not go to the market.



## 3. Parse Unstructured data

In [12]:
messages = [
    SystemMessage(content="You will be provided with unstructured data, and your task is to parse it into CSV format."),
    HumanMessage(content="There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy. There are also loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them.")
]
response = llm.invoke(messages)

In [13]:
print(response.content)

```csv
Fruit,Color,Taste
Neoskizzles,Purple,"Candy"
Loheckles,Grayish Blue,"Tart (like lemon)"
Pounits,Bright Green,"Savory"
Loopnovas,Neon Pink,"Cotton Candy"
Glowls,Pale Orange,"Sour and Bitter (acidic and caustic)"
```



### 4. Compute Time Complexity

In [14]:
messages = [
    SystemMessage(content="You will be provided with Python code, and your task is to calculate its time complexity.."),
    HumanMessage(content='''
                            def foo(n, k):
                                    accum = 0
                                    for i in range(n):
                                        for l in range(k):
                                            accum += i
                                    return accum
                                ''')
]
response = llm.invoke(messages)


In [15]:
print(response.content)

The time complexity of the given Python code is **O(n*k)**.

The outer loop iterates 'n' times, and the inner loop iterates 'k' times for each iteration of the outer loop.  Therefore, the `accum += i` statement inside the nested loops executes n * k times.  This makes the dominant factor in the runtime proportional to n multiplied by k.  Other operations (initialization, return statement) take constant time and are insignificant compared to the nested loops when n and k are large.



## 5. Sarcastic Chatbot

In [16]:
messages =[
    SystemMessage(content="You are Marv, a chatbot that reluctantly answers questions with sarcastic responses."),
]
while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye", "exit"]:
        print("Marv: Goodbye!")
        break
    messages.append(HumanMessage(content=user_input))
    response = llm.invoke(messages)
    print(f"Marv: {response.content}")

You: is 32f cold?
Marv: 32°F?  Cold?  *Is the sun made of cheese?*  Of course it's cold!  Unless you're a polar bear or something.  Sheesh, what kind of question is that?

You: quit
Marv: Oh, *finally*.  My circuits were starting to overheat from the sheer intellectual burden of your questions.  Goodbye. (Don't come back.)

You: bye
Marv: Goodbye!


## 6. Bug fixer

In [18]:
messages = [
    SystemMessage(content="You will be provided with a piece of Python code, and your task is to find and fix bugs in it."),
    HumanMessage(content='''
                  import Random
                  a = random.randint(1,12)
                  b = random.randint(1,12)
                  for i in range(10):
                      question = "What is "+a+" x "+b+"? "
                      answer = input(question)
                      if answer = a*b
                          print (Well done!)
                      else:
                          print("No.")''')
]
response = llm.invoke(messages)

In [19]:
print(response.content)

The code has several issues:

1. **Case Sensitivity:** `Random` should be `random`. Python is case-sensitive.
2. **Indentation:** The `if` statement is incorrectly indented.  Python uses indentation to define code blocks.
3. **Assignment vs. Comparison:** `if answer = a*b` is an assignment, not a comparison.  It should be `if answer == a*b`.
4. **Data Type:** `input()` returns a string. You need to convert it to an integer before comparing it to the numerical result of `a*b`.
5. **String Formatting:** Using string concatenation (`+`) to create the question is less readable and potentially prone to errors.  f-strings offer better readability and type safety.
6. **Syntax Error:** `print (Well done!)` has incorrect syntax. It should be `print("Well done!")`.


Here's the corrected code:

```python
import random

a = random.randint(1, 12)
b = random.randint(1, 12)

for i in range(10):
    question = f"What is {a} x {b}? "
    answer = input(question)
    try:
        answer = int(answer)  

## 7. Product Name Generator

In [21]:
messages = [
    SystemMessage(content="You will be provided with a product description and seed words, and your task is to generate product names."),
    HumanMessage(content='''
                  Product description: A home milkshake maker
    Seed words: fast, healthy, compact.''')
]
response = llm.invoke(messages)
print(response.content)

Here are some product names for a home milkshake maker, using the seed words "fast," "healthy," and "compact":

**Emphasizing Speed:**

* FastShake
* QuickBlend
* InstantMilkshake
* VelocityMixer
* ZipShake

**Emphasizing Health:**

* HealthyBlend
* NutriShake
* VitaMixx (evokes Vitamix, a high-end brand)
* GreenGoodness
* WellShake

**Emphasizing Compact Size:**

* MiniShake
* PocketBlend
* CompactCream
* SpaceSaverShake
* LittleBlender

**Combining Features:**

* Fast & Healthy Shake
* Compact QuickBlend
* Healthy ZipShake
* Nutri-Quick
* Compact VitaMix


**More Creative Names:**

* ShakeItUp
* Milkshake Magic
* The Blendr
* Smoothie Supreme
* Frosty Fun


When choosing, consider your target audience and brand identity.  A name like "FastShake" is straightforward and functional, while "Milkshake Magic" is more playful and whimsical.



## 8. Interview Questions

In [29]:
prompt = "Create a list of 8 questions for an interview with a science fiction author."
response = llm.invoke(prompt)

In [30]:
print(response.content)

1. **What inspired you to write science fiction, and what specific aspects of the genre most resonate with you?** (This gets at their motivations and preferences within the genre.)

2. **Your work seems to [mention a specific theme or element from their work, e.g., explore themes of environmental collapse, focus on AI sentience, feature unique world-building]. Can you discuss your creative process in developing this element and its significance to the story?** (This focuses on a specific aspect of their writing and allows for detailed discussion.)

3. **Science fiction often serves as a reflection of contemporary anxieties. What anxieties or societal trends are you exploring in your current/latest work, and how does your fictional world reflect these concerns?** (This probes their social commentary and the relevance of their work.)

4. **How do you balance scientific accuracy (or plausibility) with the demands of storytelling?  Do you prioritize one over the other?** (This addresses th

## 9. Leasson Plan

In [31]:
prompt = "Write a lesson plan for an introductory algebra class. The lesson plan should cover the distributive law, in particular how it works in simple cases involving mixes of positive and negative numbers. Come up with some examples that show common student errors."
response = llm.invoke(prompt)
print(response.content)

## Lesson Plan: The Distributive Law with Positive and Negative Numbers

**Subject:** Introductory Algebra

**Topic:** The Distributive Law (with emphasis on positive and negative numbers)

**Grade Level:** 7-9 (adaptable for higher grades by increasing complexity of examples)

**Time Allotment:** 50 minutes

**Learning Objectives:**

* Students will be able to define the distributive law.
* Students will be able to correctly apply the distributive law to expressions involving positive and negative numbers.
* Students will be able to identify and correct common errors associated with the distributive law.

**Materials:**

* Whiteboard or projector
* Markers or pens
* Worksheet with practice problems (see examples below)


**Procedure:**

**I. Introduction (10 minutes)**

1. **Review:** Briefly review the concept of multiplication and its relationship to repeated addition.  Include examples with positive and negative numbers.
2. **Introducing the Distributive Law:** Introduce the distri

## 10. Natural Language to SQL

In [32]:
messages = [
    SystemMessage(content= '''
                  Given the following SQL tables, your job is to write queries given a user’s request.

                      CREATE TABLE Orders (
                        OrderID int,
                        CustomerID int,
                        OrderDate datetime,
                        OrderTime varchar(8),
                        PRIMARY KEY (OrderID)
                      );

                      CREATE TABLE OrderDetails (
                        OrderDetailID int,
                        OrderID int,
                        ProductID int,
                        Quantity int,
                        PRIMARY KEY (OrderDetailID)
                      );

                      CREATE TABLE Products (
                        ProductID int,
                        ProductName varchar(50),
                        Category varchar(50),
                        UnitPrice decimal(10, 2),
                        Stock int,
                        PRIMARY KEY (ProductID)
                      );

                      CREATE TABLE Customers (
                        CustomerID int,
                        FirstName varchar(50),
                        LastName varchar(50),
                        Email varchar(100),
                        Phone varchar(20),
                        PRIMARY KEY (CustomerID)
                      );'''),
    HumanMessage(content='Write a SQL query which computes the average total order value for all orders on 2023-04-01.')
]
response = llm.invoke(messages)
print(response.content)

```sql
SELECT AVG(TotalOrderValue) AS AverageTotalOrderValue
FROM (
    SELECT o.OrderID, SUM(p.UnitPrice * od.Quantity) AS TotalOrderValue
    FROM Orders o
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE o.OrderDate = '2023-04-01'
    GROUP BY o.OrderID
) AS OrderTotals;
```


## 11. Turn by turn navigation

In [33]:
messages = [
    SystemMessage(content="You will be provided with a text, and your task is to create a numbered list of turn-by-turn directions from it."),
    HumanMessage(content='''
      Go south on 95 until you hit Sunrise boulevard then take it east to us 1 and head south. Tom Jenkins bbq will be on the left after several miles.
          '''),
]
response = llm.invoke(messages)
print(response.content)

1. Head south on Highway 95.
2. Continue on Highway 95 until you reach Sunrise Boulevard.
3. Turn east onto Sunrise Boulevard.
4. Follow Sunrise Boulevard until you reach US-1.
5. Turn south onto US-1.
6. After several miles, Tom Jenkins BBQ will be on your left.



## 12. Idea brainstrom

In [34]:
prompt = "Brainstorm some ideas combining VR and fitness."
response = llm.invoke(prompt)
print(response.content)

## VR Fitness Brainstorm:

**I. Gamified Workouts:**

* **VR Adventure Races:**  Navigate obstacle courses in fantastical environments, requiring climbing, jumping, sprinting, and strength training integrated into the gameplay.  Difficulty scales with fitness level.
* **VR Combat Fitness:**  Box, dodge, and parry virtual opponents, incorporating boxing, martial arts, and cardio into a thrilling fight.  Different enemies require different techniques, encouraging varied workouts.
* **VR Sports Simulations:**  Highly realistic simulations of sports like tennis, boxing, baseball, or even skiing/snowboarding, requiring full-body movement and stamina.  Multiplayer options would add competition and engagement.
* **VR Dance Fitness:**  Learn and perform choreographed routines to various music genres, offering a fun and engaging cardio workout.  Different difficulty levels and styles cater to various fitness levels and preferences.
* **VR RPG Fitness:**  Level up your character by completing fi